In [34]:
import requests
import pandas as pd
import time
import re
from bs4 import BeautifulSoup
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

import warnings 

warnings.filterwarnings('ignore')

## Web scraping

In [35]:
# Set headers and delay
headers = {
    "User-Agent": "MyWikipediaScraper/1.0 (https://example.com/contact) Python requests"
}

delay = 2

# Set url
base_url = 'https://en.wikipedia.org'
list_url = base_url + '/wiki/List_of_national_parks_of_Indonesia'

# Create request
response = requests.get(list_url, headers=headers)
response.raise_for_status()
soup = BeautifulSoup(response.text, 'html.parser')

In [36]:
# Parse the tables
tables = soup.find_all('table', class_='wikitable')
parks = []

for table in tables:
    rows = table.find_all('tr')

    # Flatten headers
    header_rows = [r for r in rows if r.find_all('th')]
    if len(header_rows) == 0:
        continue

    # First header
    top_headers = []
    for th in header_rows[0].find_all('th'):
        colspan = int(th.get('colspan', 1))
        text = th.get_text(strip=True)
        top_headers.extend([text]*colspan)

    # Second header
    sub_headers = []
    if len(header_rows) > 1:
        for th in header_rows[1].find_all('th'):
            text = th.get_text(strip=True)
            sub_headers.append(text)
    else:
        sub_headers = [None]*len(top_headers)

    # Combine top and sub headers
    final_headers = []
    sub_index = 0
    for h in top_headers:
        if h.lower() in ['total area']:
            final_headers.append(f"{h} {sub_headers[sub_index]}")
            sub_index += 1
        else:
            final_headers.append(h)

    # Parse table rows
    for row in rows[len(header_rows):]:
        cols = row.find_all(['td','th'])
        if not cols:
            continue

        data = [c.get_text(strip=True) if c.get_text(strip=True) != '' else None for c in cols]

        # Extract link
        link_tag = row.find('a', href=True)
        url = base_url + link_tag['href'] if link_tag and link_tag['href'].startswith('/wiki/') else None

        park = dict(zip(final_headers, data))
        park['url'] = url
        parks.append(park)

In [37]:
# Extract coordinates from each page
for i, park in enumerate(parks, start=1):
    url = park.get('url')
    if not url:
        park['latitude'] = None
        park['longitude'] = None
        continue

    try:
        r = requests.get(url, headers=headers)
        r.raise_for_status()
        soup2 = BeautifulSoup(r.text, 'html.parser')

        lat = soup2.select_one('.latitude')
        lon = soup2.select_one('.longitude')

        park['latitude'] = lat.text.strip() if lat else None
        park['longitude'] = lon.text.strip() if lon else None

    except Exception as e:
        print(f"Error: {e}")
        park["latitude"] = None
        park["longitude"] = None

    # Add time sleep
    time.sleep(delay)

# Store in the data frame
df = pd.DataFrame(parks)
df.head()

,Name,Year,Total Area km²,Total Area mi²,Marine area,International status,url,latitude,longitude
0,Alas Purwo,1992,434,168,None,None,https://en.wikipedia.org/wiki/Alas_Purwo_Natio...,8°41′S,114°28′E
1,Baluran,1980,250,96,None,None,https://en.wikipedia.org/wiki/Baluran_National...,7°50′S,114°22′E
2,Bromo Tengger Semeru,1983,503,194,None,World Network of Biosphere Reserves,https://en.wikipedia.org/wiki/Bromo_Tengger_Se...,8°1′S,112°55′E
3,Gunung Ciremai,2004,155,60,None,None,https://en.wikipedia.org/wiki/Gunung_Ciremai_N...,6°54′26″S,108°24′57″E
4,Gunung Gede Pangrango,1980,150,58,None,World Network of Biosphere Reserves,https://en.wikipedia.org/wiki/Gunung_Gede_Pang...,6°46′0″S,106°56′0″E


## Data cleaning

In [38]:
# Replace columns name
df.columns = ['name', 'year', 'total_area_km2', 'total_area_mi2', 'marine_area', 'international_status', 'url', 'latitude', 'longitude']

# Check data information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   name                  57 non-null     object
 1   year                  57 non-null     object
 2   total_area_km2        57 non-null     object
 3   total_area_mi2        57 non-null     object
 4   marine_area           9 non-null      object
 5   international_status  20 non-null     object
 6   url                   52 non-null     object
 7   latitude              52 non-null     object
 8   longitude             52 non-null     object
dtypes: object(9)
memory usage: 4.1+ KB


In [39]:
# Keep only characters in the name column
df['name'] = df['name'].str.replace(r'[^A-Za-z\s]', '', regex=True)

# Replace some characters in the total_area columns
df['total_area_km2'] = df['total_area_km2'].str.replace(r'[?,]', '', regex=True)
df['total_area_mi2'] = df['total_area_mi2'].str.replace(r'[?,]', '', regex=True)
df.head()

,name,year,total_area_km2,total_area_mi2,marine_area,international_status,url,latitude,longitude
0,Alas Purwo,1992,434,168,None,None,https://en.wikipedia.org/wiki/Alas_Purwo_Natio...,8°41′S,114°28′E
1,Baluran,1980,250,96,None,None,https://en.wikipedia.org/wiki/Baluran_National...,7°50′S,114°22′E
2,Bromo Tengger Semeru,1983,503,194,None,World Network of Biosphere Reserves,https://en.wikipedia.org/wiki/Bromo_Tengger_Se...,8°1′S,112°55′E
3,Gunung Ciremai,2004,155,60,None,None,https://en.wikipedia.org/wiki/Gunung_Ciremai_N...,6°54′26″S,108°24′57″E
4,Gunung Gede Pangrango,1980,150,58,None,World Network of Biosphere Reserves,https://en.wikipedia.org/wiki/Gunung_Gede_Pang...,6°46′0″S,106°56′0″E


In [40]:
# Convert DMS coordinates to decimals
def dms2d(s):
    if pd.isna(s) or not isinstance(s, str) or s.strip() == "":
        return None

    s = s.strip().upper()

    # Normalize symbols (° ′ ″ etc.)
    s = s.replace("′", "'").replace("″", '"')

    # Extract direction
    direction_match = re.search(r'([NSEW])', s)
    direction = direction_match.group(1) if direction_match else None
    s = re.sub(r'[NSEW]', '', s)

    # Split on non-numeric characters (° ' ")
    parts = [p for p in re.split(r'[°\'"]+', s) if p]

    # Parse degrees, minutes, seconds depending on available parts
    try:
        degrees = float(parts[0])
        minutes = float(parts[1]) if len(parts) > 1 else 0
        seconds = float(parts[2]) if len(parts) > 2 else 0
    except (IndexError, ValueError):
        return None

    # Apply conversion to decimals coordinates
    d = degrees + (minutes / 60) + (seconds / 3600)
    if direction in ('S', 'W'):
        d *= -1

    return round(d, 6)

df['latitude_dec'] = df['latitude'].apply(dms2d)
df['longitude_dec'] = df['longitude'].apply(dms2d)
df.head()

,name,year,total_area_km2,total_area_mi2,marine_area,international_status,url,latitude,longitude,latitude_dec,longitude_dec
0,Alas Purwo,1992,434,168,None,None,https://en.wikipedia.org/wiki/Alas_Purwo_Natio...,8°41′S,114°28′E,-8.683333,114.466667
1,Baluran,1980,250,96,None,None,https://en.wikipedia.org/wiki/Baluran_National...,7°50′S,114°22′E,-7.833333,114.366667
2,Bromo Tengger Semeru,1983,503,194,None,World Network of Biosphere Reserves,https://en.wikipedia.org/wiki/Bromo_Tengger_Se...,8°1′S,112°55′E,-8.016667,112.916667
3,Gunung Ciremai,2004,155,60,None,None,https://en.wikipedia.org/wiki/Gunung_Ciremai_N...,6°54′26″S,108°24′57″E,-6.907222,108.415833
4,Gunung Gede Pangrango,1980,150,58,None,World Network of Biosphere Reserves,https://en.wikipedia.org/wiki/Gunung_Gede_Pang...,6°46′0″S,106°56′0″E,-6.766667,106.933333


In [41]:
# Show rows where missing values occurred in lat/long columns
df[df['latitude_dec'].isna() | df['longitude_dec'].isna()]

,name,year,total_area_km2,total_area_mi2,marine_area,international_status,url,latitude,longitude,latitude_dec,longitude_dec
27,Moyo Satonda,2022,312,120,None,None,None,None,None,NaN,NaN
28,Mutis Timau,2024,788,304,None,None,None,None,None,NaN,NaN
31,Mamberamo Foja,2024,,,None,None,None,None,None,NaN,NaN
55,Zamrud,2016,314,121,None,None,None,None,None,NaN,NaN
56,Mount Maras,2016,168,65,None,None,None,None,None,NaN,NaN


In [42]:
# Fill the missing coordinates 
df.loc[27, ['latitude_dec', 'longitude_dec']] = [-8.145, 117.695] # Moyo satonda
df.loc[28, ['latitude_dec', 'longitude_dec']] = [-9.565, 124.236] # Mutis timau
df.loc[31, ['latitude_dec', 'longitude_dec']] = [-1.767, 137.839] # Mamberamo foja
df.loc[55, ['latitude_dec', 'longitude_dec']] = [0.674, 102.254] # Zamrud
df.loc[56, ['latitude_dec', 'longitude_dec']] = [-1.883, 105.833] # Mount maras

In [43]:
# Check missing values
df[['latitude_dec', 'longitude_dec']].isna().sum()

latitude_dec     0
longitude_dec    0
dtype: int64

## Geocoding

In [44]:
from geopy.geocoders import Nominatim
import time

In [45]:
# Set location
geolocator = Nominatim(user_agent="indonesia_province_locator")

# Get indonesian region or province
def get_province(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), language='en', addressdetails=True)
        if location and 'address' in location.raw:
            address = location.raw['address']
            return address.get('state') or address.get('region') or address.get('province')
    except Exception as e:
        print(f"Error: {lat}, {lon}: {e}")
        return None
    return None

# Apply to data frame
for i, row in df.iterrows():
    lat, lon = row['latitude_dec'], row['longitude_dec']
    if pd.notna(lat) and pd.notna(lon):
        df.loc[i, 'province'] = get_province(lat, lon)
        time.sleep(3)  

df.head()

,name,year,total_area_km2,total_area_mi2,marine_area,international_status,url,latitude,longitude,latitude_dec,longitude_dec,province
0,Alas Purwo,1992,434,168,None,None,https://en.wikipedia.org/wiki/Alas_Purwo_Natio...,8°41′S,114°28′E,-8.683333,114.466667,East Java
1,Baluran,1980,250,96,None,None,https://en.wikipedia.org/wiki/Baluran_National...,7°50′S,114°22′E,-7.833333,114.366667,East Java
2,Bromo Tengger Semeru,1983,503,194,None,World Network of Biosphere Reserves,https://en.wikipedia.org/wiki/Bromo_Tengger_Se...,8°1′S,112°55′E,-8.016667,112.916667,East Java
3,Gunung Ciremai,2004,155,60,None,None,https://en.wikipedia.org/wiki/Gunung_Ciremai_N...,6°54′26″S,108°24′57″E,-6.907222,108.415833,West Java
4,Gunung Gede Pangrango,1980,150,58,None,World Network of Biosphere Reserves,https://en.wikipedia.org/wiki/Gunung_Gede_Pang...,6°46′0″S,106°56′0″E,-6.766667,106.933333,West Java


In [46]:
# Create mapping province to big island
province_to_island = {
    # Java
    'Java': 'Java',
    'West Java': 'Java',
    'Central Java': 'Java',
    'East Java': 'Java',
    'Banten': 'Java',

    # Sumatra
    'Aceh': 'Sumatra',
    'North Sumatra': 'Sumatra',
    'West Sumatra': 'Sumatra',
    'Riau': 'Sumatra',
    'Jambi': 'Sumatra',
    'Bengkulu': 'Sumatra',
    'South Sumatra': 'Sumatra',
    'Lampung': 'Sumatra',
    'Bangka-Belitung Islands': 'Sumatra',

    # Kalimantan
    'West Kalimantan': 'Kalimantan',
    'Central Kalimantan': 'Kalimantan',
    'South Kalimantan': 'Kalimantan',
    'East Kalimantan': 'Kalimantan',
    'North Kalimantan': 'Kalimantan',
    'Sarawak': 'Kalimantan',

    # Sulawesi
    'Sulawesi': 'Sulawesi',
    'North Sulawesi': 'Sulawesi',
    'Central Sulawesi': 'Sulawesi',
    'South Sulawesi': 'Sulawesi',
    'Southeast Sulawesi': 'Sulawesi',
    'Gorontalo': 'Sulawesi',
    'West Sulawesi': 'Sulawesi',

    # Bali and Nusa Tenggara
    'Bali': 'Bali and Nusa Tenggara',
    'West Nusa Tenggara': 'Bali and Nusa Tenggara',
    'East Nusa Tenggara': 'Bali and Nusa Tenggara',

    # Maluku
    'Maluku': 'Maluku',
    'North Maluku': 'Maluku',

    # Papua
    'Papua': 'Papua',
    'West Papua': 'Papua',
    'South Papua': 'Papua',
    'Central Papua': 'Papua',
    'Highland Papua': 'Papua',
}

df['island'] = df['province'].map(province_to_island)
df.head()

,name,year,total_area_km2,total_area_mi2,marine_area,international_status,url,latitude,longitude,latitude_dec,longitude_dec,province,island
0,Alas Purwo,1992,434,168,None,None,https://en.wikipedia.org/wiki/Alas_Purwo_Natio...,8°41′S,114°28′E,-8.683333,114.466667,East Java,Java
1,Baluran,1980,250,96,None,None,https://en.wikipedia.org/wiki/Baluran_National...,7°50′S,114°22′E,-7.833333,114.366667,East Java,Java
2,Bromo Tengger Semeru,1983,503,194,None,World Network of Biosphere Reserves,https://en.wikipedia.org/wiki/Bromo_Tengger_Se...,8°1′S,112°55′E,-8.016667,112.916667,East Java,Java
3,Gunung Ciremai,2004,155,60,None,None,https://en.wikipedia.org/wiki/Gunung_Ciremai_N...,6°54′26″S,108°24′57″E,-6.907222,108.415833,West Java,Java
4,Gunung Gede Pangrango,1980,150,58,None,World Network of Biosphere Reserves,https://en.wikipedia.org/wiki/Gunung_Gede_Pang...,6°46′0″S,106°56′0″E,-6.766667,106.933333,West Java,Java


## Data visualization

In [47]:
# Count national parks per island
island_counts = df['island'].value_counts().reset_index()
island_counts.columns = ['island', 'count']

# Sort smallest → largest
island_counts = island_counts.sort_values('count', ascending=False)

fig = px.bar(
    island_counts,
    x='count',
    y='island',
    orientation='h',
    color='island',
    title='National Parks Count by Island'
)

fig.update_layout(
    xaxis_title="Number of National Parks",
    yaxis_title="Island",
    title_x=0.5,              
    title_font=dict(size=22),
    width=1200,
    height=600,
)

fig.show()

In [48]:
# Create a pie chart
fig = px.pie(
    df,
    names="island",
    title="Proportion of National Parks by Island",
    hole=0.1, 
)

fig.update_traces(textposition='inside', textinfo='percent+label')

fig.update_layout(
    title={
        "x": 0.45,
        "xanchor": "center",
        "font": {"size": 18}
    },
    width=700,
    height=500
)

fig.show()

In [ ]:
# Create map visualization
fig = px.scatter_mapbox(
    df,
    lat="latitude_dec",
    lon="longitude_dec",
    color="island",
    hover_name="name",
    hover_data={"island": True, "province": True,  "latitude_dec": False, "longitude_dec": False},
    zoom=4,
    height=600
)

# Build dropdown buttons
buttons = []
for trace in fig.data:
    island_name = trace.name
    visibility = [(t.name == island_name) for t in fig.data]

    buttons.append(
        dict(
            label=island_name,
            method="restyle",
            args=[{"visible": visibility}]
        )
    )

# Add show all
buttons.insert(
    0,
    dict(
        label="Show All",
        method="restyle",
        args=[{"visible": [True] * len(fig.data)}]
    )
)

fig.update_layout(
        title={
        "text": "National Parks of Indonesia",
        "y": 0.95,
        "x": 0.5,       
        "xanchor": "center",
        "yanchor": "top",   
        "font": {"size": 18}
        },
    width=1400,
    height=600,
    mapbox_style="carto-positron",
    mapbox_center={"lat": -2, "lon": 118},
    legend_title_text="Island",
    updatemenus=[
        dict(
            buttons=buttons,
            direction="down",
            showactive=True,
            x=-0.01,
            y=1
        )
    ]
)

fig.show()